In [1]:
from datasets import list_datasets, load_dataset
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import classification_report
from tqdm.notebook import tqdm as tqdm
#from pymagnitude import *
from nltk import word_tokenize
import pandas as pd
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import Dataset, DataLoader
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

In [10]:
from pymagnitude import *

ModuleNotFoundError: No module named 'pymagnitude'

In [2]:
tags = [('tamilmixsentiment', None, 'tamil'), 
        ('offenseval_dravidian', 'tamil', 'tamil'), 
        ('offenseval_dravidian', 'malayalam', 'malayalam'),
        ('offenseval_dravidian', 'kannada', 'kannada'),
        ('kan_hope', None, 'kannada'),
        #('Shushant/NepaliSentiment', None, 'hindi')
        ]

In [3]:
def get_data(tag):
    dataset = load_dataset(tag[0], tag[1])
    train_df = pd.DataFrame()
    val_df = pd.DataFrame()

    train_df['text'] = dataset['train']['text']
    train_df['label'] = dataset['train']['label']
    
    val_df['text'] = dataset['validation']['text']
    val_df['label'] = dataset['validation']['label']
    
#     print("TRAIN DESCRIPTION:")
#     print("Value Counts:")
#     print(train_df['label'].value_counts())
          
#     print("Sample text and label:")
#     for i in range(5):
#         idx = random.randint(0, len(train_df))
#         sample = train_df.iloc[idx]
#         print("Text: {}, Label:{}".format(sample['text'], sample['label']))
    return train_df, val_df

In [4]:
class LinearNetwork(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        # define layers
        self.fc1 = nn.Linear(in_features=input_size,out_features=1024)
        self.fc2 = nn.Linear(in_features=1024,out_features=output_size)
    # define forward function
    def forward(self, t):
        # fc 1
        t=self.fc1(t)
        t=F.relu(t)
        # fc 2
        t=self.fc2(t)
        # don't need softmax here since we'll use cross-entropy as activation.
        return t

In [5]:
def get_embedding(text, model, model_type):
    if model_type in ['TfidfVectorizer', 'CountVectorizer']:
        return model.transform(text).toarray()
    elif model_type == 'BERT':
        print("BERT model embedding...")
        return model.encode(text, batch_size=8, show_progress_bar=True)
    elif model_type == 'magnitude':
        vectors = []
        for sentence in tqdm(text):
            vectors.append(np.average(model.query(word_tokenize(sentence)), axis=0))
        return vectors    

In [7]:
class CodemixDataset(Dataset):
    def __init__(self, df, encoder_model, encoding_type):
        self.df = df
        self.embedding = get_embedding(list(self.df['text']), \
                                      encoder_model, encoding_type)
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        text_embedding = self.embedding[idx]
        label = self.df.iloc[idx]['label']
        sample = {
            'text': text_embedding,
            'label': label
        }
        return sample

In [16]:
def get_scores(y_true, y_pred):
    if torch.cuda.is_available():
        accuracy = accuracy_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred, average='weighted')
        recall = recall_score(y_true, y_pred, average='weighted')
        precision = precision_score(y_true, y_pred, average='weighted')
    else:
        accuracy = accuracy_score(y_true.cpu(), y_pred.cpu())
        f1 = f1_score(y_true.cpu(), y_pred.cpu(), average='weighted')
        recall = recall_score(y_true.cpu(), y_pred.cpu(), average='weighted')
        precision = precision_score(y_true.cpu(), y_pred.cpu(), average='weighted')
    return accuracy, f1, recall, precision

In [17]:
def train(model_architecture, encoder_model, encoding_type, train_df, test_df, epochs, \
          learning_rate, batch_size, log, device):
    input_size = get_embedding([train_df.iloc[0]['text']], encoder_model, encoding_type).shape
    model = model_architecture(input_size[1], len(train_df['label'].unique())).to(device)
    
    train_set = CodemixDataset(train_df, encoder_model, encoding_type)
    val_set = CodemixDataset(val_df, encoder_model, encoding_type)
    train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(val_set, batch_size=batch_size, shuffle=False)

    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = torch.nn.CrossEntropyLoss()

    model.train()
    for epoch in range(epochs):
        output_labels = torch.Tensor().to(device)
        true_labels = torch.Tensor().to(device)
        total_loss = 0
        for data in tqdm(train_loader):
            text, labels = data['text'], data['label']
            inputs = text.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs.type(torch.float))
            predicted_labels = torch.argmax(outputs, dim=1)
            #check this
            output_labels = torch.cat((output_labels, predicted_labels), 0)
            true_labels = torch.cat((true_labels, labels), 0)
            loss = criterion(outputs, labels.to(device))
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
        tr_accuracy, tr_f1, tr_recall, tr_precision = get_scores(true_labels, output_labels)
        net_train_loss = total_loss/(len(train_loader)*batch_size)
        print('Epoch: {}\t Train Loss: {:.4f} \t Train F1:{:.2f}'.format(epoch, net_train_loss, tr_f1), end='\t')
        
        output_labels = torch.Tensor().to(device)
        true_labels = torch.Tensor().to(device)
        total_loss = 0
        for i, data in enumerate(test_loader, 0):
            text, labels = data['text'], data['label']
            
            inputs = text.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs.type(torch.float))
            predicted_labels = torch.argmax(outputs, dim=1)
            output_labels = torch.cat((output_labels, predicted_labels), 0)
            true_labels = torch.cat((true_labels, labels), 0)
            loss = criterion(outputs, labels)

            total_loss += loss.item()
        ts_accuracy, ts_f1, ts_recall, ts_precision = get_scores(true_labels, output_labels)
        net_test_loss = total_loss/(len(train_loader)*batch_size)
        print('Test Loss: {:.4f} \t Test F1:{:.2f}'.format(net_test_loss, ts_f1))

        if log != None:
            log({
                    "train accuracy": tr_accuracy,
                    "train f1": tr_f1,
                    "train recall": tr_recall,
                    "train precision": tr_precision,
                    
                    "test accuarcy": ts_accuracy,
                    "test f1": ts_f1,
                    "test recall": ts_recall,
                    "test precision": ts_precision,
                    
                    "train loss": net_train_loss,
                    "test loss": net_test_loss
                    })

In [14]:
EPOCHS = 10
ETA = 0.001
BATCH_SIZE = 32
tdevice = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_df, val_df = get_data(tags[1])
# train_df = pd.read_csv('../dataset/dravidian-codemix/tamil_train.tsv', sep='\t')
# val_df = pd.read_csv('../dataset/dravidian-codemix/tamil_dev.tsv', sep='\t')

# tfidf vec
vectorizer = CountVectorizer(analyzer='char', ngram_range=(1,3), max_features=2048)
vectorizer.fit(train_df['text'])
train(LinearNetwork, vectorizer, 'CountVectorizer', train_df, val_df, epochs=EPOCHS, \
      learning_rate=ETA, batch_size=BATCH_SIZE, log=None, device=tdevice)

# # tfidf vec
# vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(1,3), max_features=2048)
# vectorizer.fit(train_df['text'])
# train(LinearNetwork, vectorizer, 'TfidfVectorizer', train_df, val_df, epochs=EPOCHS, \
#       learning_rate=ETA, batch_size=BATCH_SIZE, log=None, device=tdevice)

# #fasttext - specific language
# model = Magnitude("../weights/fasttext/{}/{}.magnitude".format(tags[0][2], tags[0][2]))
# train(LinearNetwork, model, 'magnitude', train_df, val_df, epochs=EPOCHS, \
#       learning_rate=ETA, batch_size=BATCH_SIZE, log=None, device=tdevice)

# bert - multilingual
# model = SentenceTransformer('distiluse-base-multilingual-cased')
# train(LinearNetwork, model, 'BERT', train_df, val_df, epochs=EPOCHS, \
#       learning_rate=ETA, batch_size=BATCH_SIZE, log=None, device=tdevice)



Reusing dataset offenseval_dravidian (C:\Users\deepa\.cache\huggingface\datasets\offenseval_dravidian\tamil\1.0.0\caf62757ff7f5922e043f21abf68745096b24007c4b79d5b2344ea3a7238563f)


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1099 [00:00<?, ?it/s]

Epoch: 0	 Train Loss: 0.0244 	 Train F1:0.70	Test Loss: 0.0027 	 Test F1:0.71


C:\Users\deepa\anaconda3\envs\nn_project\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

Epoch: 7	 Train Loss: 0.0033 	 Train F1:0.97	Test Loss: 0.0051 	 Test F1:0.73


  0%|          | 0/1099 [00:00<?, ?it/s]

Epoch: 8	 Train Loss: 0.0022 	 Train F1:0.98	Test Loss: 0.0061 	 Test F1:0.73


  0%|          | 0/1099 [00:00<?, ?it/s]

Epoch: 9	 Train Loss: 0.0021 	 Train F1:0.98	Test Loss: 0.0058 	 Test F1:0.73


In [10]:
tdevice = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tdevice

device(type='cuda')

In [15]:
torch.cuda.is_available()

True

In [6]:
import torch
print(torch.__version__)

1.11.0+cpu
